# No more witch's brew: 
# Simplify database security management with Oracle 23ai

Flora Barriele Lift Database
Implementation Technical Lead at Oracle


# Bag of snakes...?

- 100+ tickets for single SELECT privilege on new tables?
- One app can write on the DB, the other one shouldn't?
- Audit trail growth becoming out of control?

# ... or can or worms?

- Home-made pale imitation of SQL firewall?


# Schema privileges

- Grant privilege on ALL objects in a schema
- To both current and future objects in the schema


In [ ]:
%%bash

run/SQL application_pdb_admin "

create user APPUSER identified by \"VeryStrong!Password#123\";
grant CREATE SESSION to APPUSER;

"

In [ ]:
%%bash

run/SQL application_pdb_APPUSER "

select * from SPELLS.LIST;

"

In [ ]:
%%bash

run/SQL application_pdb_admin "

grant select any table on schema SPELLS to APPUSER;
grant update any table on schema SPELLS to APPUSER;
grant insert any table on schema SPELLS to APPUSER;

"

In [ ]:
%%bash

run/SQL application_pdb_APPUSER "

select * from SPELLS.LIST where ID<10;

"

In [ ]:
%%bash

run/SQL application_pdb_admin "

CREATE TABLE spells.potency (
    spell_id NUMBER NOT NULL,
    spell_level NUMBER NOT NULL,
    CONSTRAINT fk_spell
        FOREIGN KEY (spell_id)
        REFERENCES spells.list(id)
);


INSERT INTO spells.potency (spell_id, spell_level)
SELECT id, 1
FROM spells.list;

commit;

"

In [ ]:
%%bash

run/SQL application_pdb_APPUSER "

select * from SPELLS.potency where spell_id<10;

"

# Read-only users
- For a local user
- Acts like PDB is opened in read-only mode

In [ ]:
%%bash

run/SQL application_pdb_APPUSER "

insert into SPELLS.LIST(name) values ('Luck');
commit;
select * from SPELLS.LIST where name like '%uck%';

"

In [ ]:
%%bash

run/SQL application_pdb_admin "

alter user APPUSER read only;

"

In [ ]:
%%bash

run/SQL application_pdb_APPUSER "

select * from SPELLS.LIST where name like '%uck%';
insert into SPELLS.LIST(name) values ('Sobriety');

"

In [ ]:
%%bash

run/SQL application_pdb_admin "

alter user APPUSER read write;

"

In [ ]:
%%bash

run/SQL application_pdb_APPUSER "

insert into SPELLS.LIST(name) values ('Inebriety');
commit;
select * from SPELLS.LIST where name like '%briety%';

"

# Hybrid Read-Only Mode for PDB

Hybrid open mode where PDB operates as
- read-only for local users
- read-write for common users

In [ ]:
%%bash

run/SQL application_cdb_admin "

alter pluggable database APPLICATION_PDB close immediate;
alter pluggable database APPLICATION_PDB open hybrid read only;

"

In [ ]:
%%bash

run/SQL application_pdb_APPUSER "

select * from SPELLS.LIST where name like '%ind%';

"

In [ ]:
%%bash

run/SQL application_pdb_APPUSER "

insert into SPELLS.LIST(name) values ('Kindness');

"

In [ ]:
%%bash

run/SQL application_pdb_admin "

insert into SPELLS.LIST(name) values ('Kindness');
commit;

"

In [ ]:
%%bash

run/SQL application_cdb_admin "

alter pluggable database APPLICATION_PDB close immediate;
alter pluggable database APPLICATION_PDB open ;

"

In [ ]:
%%bash

run/SQL application_pdb_APPUSER "

insert into SPELLS.LIST(name) values ('Madness');
commit;
select * from SPELLS.LIST where name like '%ness%';

"

# Audit Object Actions at Column Level
- Audit individual columns in tables and views

In [ ]:
%%bash

run/SQL application_pdb_admin "

BEGIN
DBMS_AUDIT_MGMT.CLEAN_AUDIT_TRAIL(
audit_trail_type         =>  DBMS_AUDIT_MGMT.AUDIT_TRAIL_UNIFIED,
use_last_arch_timestamp  =>  FALSE);
END;
/

"

In [ ]:
%%bash

run/SQL application_pdb_admin "

SELECT EVENT_TIMESTAMP, DBUSERNAME, ACTION_NAME, OBJECT_SCHEMA, OBJECT_NAME, SQL_TEXT
FROM   UNIFIED_AUDIT_TRAIL
WHERE  OBJECT_NAME = 'LIST'
ORDER BY EVENT_TIMESTAMP;	

"

In [ ]:
%%bash

run/SQL application_pdb_admin "

CREATE AUDIT POLICY spells_audit_policy 
ACTIONS SELECT(name) ON SPELLS.LIST;

AUDIT POLICY spells_audit_policy;

"

In [ ]:
%%bash

run/SQL application_pdb_APPUSER "

SELECT id FROM SPELLS.LIST where id<10;
SELECT name FROM SPELLS.LIST where id<10;

"

In [ ]:
%%bash

run/SQL application_pdb_admin "

SELECT EVENT_TIMESTAMP, DBUSERNAME, ACTION_NAME, OBJECT_SCHEMA, OBJECT_NAME, SQL_TEXT
FROM   UNIFIED_AUDIT_TRAIL
WHERE  OBJECT_NAME = 'LIST'
ORDER BY EVENT_TIMESTAMP;

"

# SQL Firewall

Inspects all incoming SQL statements and ensures:

- only explicitly authorized SQL is run
- only session with specific context can connect


![SQLFIREWALL](graphics/sql_firewall.png) 

In [ ]:
%%bash

run/SQL application_pdb_admin "

show con_name
create user FIREFIGHTER identified by \"V3ry!Str0ng!\";
grant CREATE SESSION to FIREFIGHTER ;
grant SQL_FIREWALL_ADMIN to FIREFIGHTER ;

"

In [ ]:
%%bash

run/SQL application_pdb_firefighter "

exec DBMS_SQL_FIREWALL.ENABLE;
exec DBMS_SQL_FIREWALL.CREATE_CAPTURE ('APPUSER');
select * from DBA_SQL_FIREWALL_CAPTURES ;

"

In [ ]:
%%bash

run/SQL_from_app_server1 application_pdb_APPUSER "

select * from SPELLS.LIST where name like '%iety%' ;
insert into SPELLS.LIST(name) values ('Control');
commit;

"

In [ ]:
%%bash

run/SQL application_pdb_firefighter "

exec DBMS_SQL_FIREWALL.STOP_CAPTURE ('APPUSER');
select * from DBA_SQL_FIREWALL_CAPTURES ;

"

In [ ]:
%%bash

run/SQL application_pdb_firefighter "

select USERNAME, SQL_TEXT, CLIENT_PROGRAM, IP_ADDRESS from DBA_SQL_FIREWALL_CAPTURE_LOGS ;

"

In [ ]:
%%bash

run/SQL application_pdb_firefighter "

select * from DBA_SQL_FIREWALL_ALLOW_LISTS ;

"

In [ ]:
%%bash

run/SQL application_pdb_firefighter "

exec DBMS_SQL_FIREWALL.GENERATE_ALLOW_LIST ('APPUSER');
select USERNAME, GENERATED_ON, STATUS, ENFORCE, BLOCK from DBA_SQL_FIREWALL_ALLOW_LISTS ;

"

In [ ]:
%%bash

run/SQL application_pdb_firefighter "

select * from DBA_SQL_FIREWALL_ALLOWED_IP_ADDR;
select * from DBA_SQL_FIREWALL_ALLOWED_OS_PROG;
select * from DBA_SQL_FIREWALL_ALLOWED_OS_USER;

"

In [ ]:
%%bash

run/SQL application_pdb_firefighter "

select USERNAME, SQL_TEXT from DBA_SQL_FIREWALL_ALLOWED_SQL ;

"

In [ ]:
%%bash

run/SQL application_pdb_firefighter "

BEGIN
  DBMS_SQL_FIREWALL.ENABLE_ALLOW_LIST (
    username       => 'APPUSER',
    enforce        => DBMS_SQL_FIREWALL.ENFORCE_CONTEXT,
    block          => TRUE
   );
END;
/

"

In [ ]:
%%bash

run/SQL_from_app_server1 application_pdb_APPUSER "

update SPELLS.LIST set name='Control freak' where name='Control';
commit;

"

In [ ]:
%%bash

run/SQL_from_app_server2 application_pdb_APPUSER "

select * from SPELLS.LIST where name like '%iety%' ;

"

In [ ]:
%%bash

run/SQL application_pdb_firefighter "

exec DBMS_SQL_FIREWALL.DISABLE_ALLOW_LIST ('APPUSER');

"

In [ ]:
%%bash

run/SQL_from_app_server2 application_pdb_APPUSER "

select * from SPELLS.LIST where name like '%iety%' ;

"

In [ ]:
%%bash

run/SQL application_pdb_firefighter "

BEGIN
  DBMS_SQL_FIREWALL.ENABLE_ALLOW_LIST (
    username       => 'APPUSER',
    enforce        => DBMS_SQL_FIREWALL.ENFORCE_SQL,
    block          => TRUE
   );
END;
/

"

In [ ]:
%%bash

run/SQL application_pdb_firefighter "

select USERNAME, SQL_TEXT from DBA_SQL_FIREWALL_ALLOWED_SQL ;

"

In [ ]:
%%bash

run/SQL_from_app_server1 application_pdb_APPUSER "

update SPELLS.LIST set name='Control freak' where name='Control';

"

In [ ]:
%%bash

run/SQL application_pdb_firefighter "

exec DBMS_SQL_FIREWALL.DISABLE_ALLOW_LIST ('APPUSER');

BEGIN
  DBMS_SQL_FIREWALL.ENABLE_ALLOW_LIST (
    username       => 'APPUSER',
    enforce        => DBMS_SQL_FIREWALL.ENFORCE_ALL,
    block          => TRUE
   );
END;
/

"

In [ ]:
%%bash

run/SQL_from_app_server2 application_pdb_APPUSER ""

In [ ]:
%%bash

run/SQL_from_app_server1 application_pdb_APPUSER "

select * from SPELLS.LIST where name like '%iety%' ;

"

In [ ]:
%%bash

run/SQL_from_app_server1 application_pdb_APPUSER "

update SPELLS.LIST set name='Control freak' where name='Control';

"

# SQL Firewall
Integration with Data Safe on OCI

![SQLFIREWALL](graphics/data_safe.png)

In [ ]:
%%bash

run/SQL application_pdb_firefighter "

select USERNAME, GENERATED_ON, STATUS, ENFORCE, BLOCK from DBA_SQL_FIREWALL_ALLOW_LISTS ;

"

# "Best soups are made in old pots" => NO!

Improve security efficiency and security management thanks to Oracle 23ai!

# Thanks!

- E-Mail: flora.barriele@oracle.com
- TwittX: @floo_bar
- LinkedIn: florabarriele